<a href="https://colab.research.google.com/github/venkatmakam/unsloth_pynb/blob/main/qwen_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Upload**

In [1]:
from google.colab import files
uploaded = files.upload()

Saving capl_to_busmaster_train.jsonl to capl_to_busmaster_train.jsonl


**2. Install**

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

**3. Train**

In [3]:
#!/usr/bin/env python3
"""
Fixed training script with proper BUSMaster C++ API examples
"""

import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset

def format_instruction(sample):
    """STRICT instruction format - no explanations allowed"""
    return {
        "text": f"""<|im_start|>system
CONVERT CAPL TO BUSMASTER C++.
RULES:
1. Generate ONLY code - NO explanations, NO comments, NO text
2. Follow EXACT conversion pattern from examples
3. Output ONLY the converted busmaster C++ code<|im_end|>
<|im_start|>user
{sample['input']}<|im_end|>
<|im_start|>assistant
{sample['output']}<|im_end|>"""
    }
def main():
    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Qwen2.5-Coder-7B-bnb-4bit",
        max_seq_length=2048,
        load_in_4bit=True,
    )

    # Setup LoRA
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
    )

    # Prepare data
    #dataset = Dataset.from_list(create_training_data())

    dataset = load_dataset("json", data_files="capl_to_busmaster_train.jsonl")
    dataset = dataset.map(format_instruction)
    temp_dataset = dataset["train"]
    # Show training examples
    # print("Training examples:")
    # for i, example in enumerate(dataset[:3]):
    #     print(f"\nExample {i+1}:")
    #     print(example["text"])

    # Train
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=temp_dataset,
        dataset_text_field="text",
        max_seq_length=2048,
        args=TrainingArguments(
            output_dir="./capl-to-busmaster",
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            learning_rate=2e-4,
            num_train_epochs=5,  # Increased for better learning
            logging_steps=10,
            save_steps=500,
            optim="adamw_8bit",
            seed=3407,
            fp16=not torch.cuda.is_bf16_supported(),
            bf16=torch.cuda.is_bf16_supported(),
        ),
    )

    print("\nStarting training...")
    trainer.train()

    # Save model
    print("Saving model...")
    model.save_pretrained_merged(
        "./capl-to-busmaster-merge",
        tokenizer,
        save_method="merged_16bit",
    )

    print("Training completed!")

if __name__ == "__main__":
    main()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [05:03<00:00, 75.97s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [05:54<00:00, 88.61s/it]


Unsloth: Merge process complete. Saved to `/content/capl-to-busmaster-merge`
Training completed!


**4. Save Model**

In [4]:
#!/usr/bin/env python3
"""
Create STRICT Ollama Modelfile - no explanations allowed
"""

import os
import shutil

def create_strict_modelfile():
    """Create Modelfile that forces direct conversion only"""

    system_prompt = """CONVERT CAPL TO BUSMASTER C++.
APPEND _CAPL TO ALL CAPL FUNCTION NAMES.
USE BUSMASTER API: Trace(), SendMsg(), SetTimer(), OnMessage_*, OnTimer_*, etc.
OUTPUT ONLY THE CONVERTED C++ CODE.
NO EXPLANATIONS. NO COMMENTS. NO TEXT.
ONLY CODE."""

    modelfile_content = f"""FROM ./model

TEMPLATE \"\"\"<|im_start|>system
{{{{ .System }}}}
<|im_end|>
<|im_start|>user
{{{{ .Prompt }}}}
<|im_end|>
<|im_start|>assistant
\"\"\"

SYSTEM \"\"\"{system_prompt}\"\"\"

PARAMETER temperature 0.1
PARAMETER top_p 0.7
PARAMETER num_predict 512
"""

    ollama_dir = "./capl-busmaster-strict-ollama"
    os.makedirs(ollama_dir, exist_ok=True)

    with open(f"{ollama_dir}/Modelfile", "w") as f:
        f.write(modelfile_content)

    # Copy model files
    model_path = "./capl-to-busmaster-merge"
    if os.path.exists(model_path):
        model_files_dir = f"{ollama_dir}/model"
        os.makedirs(model_files_dir, exist_ok=True)

        for item in os.listdir(model_path):
            src = os.path.join(model_path, item)
            dst = os.path.join(model_files_dir, item)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
            else:
                shutil.copytree(src, dst, dirs_exist_ok=True)

    print(f"✅ STRICT Ollama setup complete in: {ollama_dir}")
    return ollama_dir

def main():
    ollama_dir = create_strict_modelfile()

    print(f"\n🎯 STRICT Model Ready:")
    print(f"1. cd {ollama_dir}")
    print(f"2. ollama create capl-strict -f Modelfile")
    print(f"3. Test: ollama run capl-strict \"doubleVal = atodbl(\\\"3.14\\\"); strVal = ltoa(123); len = strlen(\\\"hello\\\");\"")
    print(f"   Expected: doubleVal = atodbl_CAPL(\\\"3.14\\\"); strVal = ltoa_CAPL(123); len = strlen_CAPL(\\\"hello\\\");")

if __name__ == "__main__":
    main()

✅ STRICT Ollama setup complete in: ./capl-busmaster-strict-ollama

🎯 STRICT Model Ready:
1. cd ./capl-busmaster-strict-ollama
2. ollama create capl-strict -f Modelfile
3. Test: ollama run capl-strict "doubleVal = atodbl(\"3.14\"); strVal = ltoa(123); len = strlen(\"hello\");"
   Expected: doubleVal = atodbl_CAPL(\"3.14\"); strVal = ltoa_CAPL(123); len = strlen_CAPL(\"hello\");
